In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
df = pd.read_csv("final.csv")

C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\4268510692.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("final.csv")


In [3]:
df

,Unnamed: 0,Year,FlightDate,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,19038,2024,2024-01-15,20409,N945JT,616,14771,"San Francisco, CA",91,12478,...,10.6,7.2,15.0,0.0,NaN,54.0,9.0,NaN,1020.4,NaN
1,19039,2024,2024-01-28,19805,N990AU,341,11298,"Dallas/Fort Worth, TX",74,14492,...,7.8,1.1,16.7,0.0,0.0,282.0,10.1,NaN,1028.9,NaN
2,19040,2024,2024-01-02,19393,N7884G,3257,10821,"Baltimore, MD",35,14107,...,5.5,2.8,7.8,0.0,0.0,310.0,15.4,NaN,1019.9,NaN
3,19041,2024,2024-01-07,20304,N311SY,3848,13487,"Minneapolis, MN",63,13232,...,-4.3,-5.0,-1.7,0.0,0.0,317.0,10.2,NaN,1015.0,NaN
4,19042,2024,2024-01-31,20452,N642RW,3493,12264,"Washington, DC",38,10785,...,4.9,2.8,6.7,0.0,0.0,33.0,6.8,NaN,1019.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546943,548681,2022,2022-12-20,20363,N293PQ,5136,10397,"Atlanta, GA",34,14814,...,0.9,-7.2,12.8,0.3,0.0,26.0,7.3,NaN,1020.1,NaN
546944,548682,2022,2022-12-07,20416,N649NK,144,13796,"Oakland, CA",91,12889,...,12.1,10.6,13.9,0.3,0.0,223.0,8.7,NaN,1021.9,NaN
546945,548683,2022,2022-12-10,19393,N492WN,3183,10693,"Nashville, TN",54,11259,...,5.7,4.4,7.2,5.8,0.0,1.0,2.2,NaN,1018.4,NaN
546946,548684,2022,2022-12-21,19393,N8727M,1364,12402,"Hilo, HI",2,12173,...,-16.4,-25.6,10.6,6.4,127.0,35.0,20.2,NaN,1029.2,NaN


In [7]:
df = df.drop(['Unnamed: 0', 'Year', 'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'DivAirportLandings', 'wpgt', 'tsun'], axis = 1)

In [9]:
df = df.drop(['index'], axis = 1)
df.columns

Index(['FlightDate', 'DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'snow',
       'wdir', 'wspd', 'pres'],
      dtype='object')

In [11]:
grouped = df.groupby(['FlightDate', 'OriginCityName'])

# Fill NaN in Col1 with the mean of matching groups
for col in ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']:
    df[col] = grouped[col].transform(lambda x: x.fillna(x.mean()))

In [12]:
df.isna().sum()

FlightDate                              0
DOT_ID_Reporting_Airline                0
Tail_Number                          1989
Flight_Number_Reporting_Airline         0
OriginAirportID                         0
OriginCityName                          0
OriginWac                               0
DestAirportID                           0
DestCityName                            0
DestWac                                 0
CRSDepTime                              0
DepTime                              9945
DepDelayMinutes                      9953
CRSArrTime                              0
ArrTime                             10419
ArrDelayMinutes                     11676
Diverted                                0
CRSElapsedTime                          0
ActualElapsedTime                   11676
AirTime                             11676
Flights                                 0
Distance                                0
CarrierDelay                       431997
WeatherDelay                      

In [13]:
df = df.drop(['snow'], axis=1)
df = df.dropna()
df

,FlightDate,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,DestCityName,DestWac,...,SecurityDelay,LateAircraftDelay,CancellationCode_encoded,tavg,tmin,tmax,prcp,wdir,wspd,pres
4,2024-01-31,20452,N642RW,3493,12264,"Washington, DC",38,10785,"Burlington, VT",16,...,0.0,0.0,0,4.9,2.8,6.7,0.0,33.0,6.8,1019.2
5,2024-01-15,19790,N590NW,1204,11433,"Detroit, MI",43,14635,"Fort Myers, FL",33,...,0.0,58.0,0,-15.3,-18.5,-11.9,0.0,234.0,21.8,1023.6
7,2024-01-05,20368,333NV,292,14262,"Palm Springs, CA",91,10666,"Bellingham, WA",93,...,0.0,1.0,0,13.8,5.6,21.7,0.0,326.0,8.5,1020.4
10,2024-01-13,19790,N921AT,2459,12197,"White Plains, NY",22,14027,"West Palm Beach/Palm Beach, FL",33,...,0.0,0.0,0,7.9,0.6,13.9,35.3,226.0,21.8,995.3
17,2024-01-09,19393,N255WN,1670,14986,"Sarasota/Bradenton, FL",33,11259,"Dallas, TX",74,...,0.0,366.0,0,21.2,17.2,25.0,7.2,178.0,30.8,1010.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546904,2022-12-24,20304,N730EV,3226,13930,"Chicago, IL",41,12197,"White Plains, NY",22,...,0.0,0.0,0,22.9,20.0,27.0,0.2,186.0,16.9,1015.3
546907,2022-12-01,20397,N592NN,5087,11278,"Washington, DC",38,13198,"Kansas City, MO",64,...,0.0,21.0,0,-7.5,-20.5,4.4,4.1,237.0,29.2,1002.0
546917,2022-12-29,20416,N921NK,2410,11618,"Newark, NJ",21,13796,"Oakland, CA",91,...,0.0,14.0,0,19.7,16.6,23.1,0.0,43.0,17.1,1023.8
546921,2022-12-23,19790,N897DN,2554,12892,"Los Angeles, CA",91,13303,"Miami, FL",33,...,0.0,32.0,0,23.1,21.0,27.0,0.1,170.0,14.1,1021.4


In [14]:
df.isna().sum()

FlightDate                         0
DOT_ID_Reporting_Airline           0
Tail_Number                        0
Flight_Number_Reporting_Airline    0
OriginAirportID                    0
OriginCityName                     0
OriginWac                          0
DestAirportID                      0
DestCityName                       0
DestWac                            0
CRSDepTime                         0
DepTime                            0
DepDelayMinutes                    0
CRSArrTime                         0
ArrTime                            0
ArrDelayMinutes                    0
Diverted                           0
CRSElapsedTime                     0
ActualElapsedTime                  0
AirTime                            0
Flights                            0
Distance                           0
CarrierDelay                       0
WeatherDelay                       0
NASDelay                           0
SecurityDelay                      0
LateAircraftDelay                  0
C

In [15]:
df[['ArrDelayMinutes', 'Diverted', 'ActualElapsedTime', 'AirTime', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'CancellationCode_encoded']] = df[['ArrDelayMinutes', 'Diverted', 'ActualElapsedTime', 'AirTime', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'CancellationCode_encoded']].fillna(0)

In [16]:
df['DateObjects'] = pd.to_datetime(df['FlightDate'])

In [17]:
df = df.sort_values(by='DateObjects')

In [18]:
df = df.drop(['FlightDate'], axis = 1)

In [23]:
def time_to_minutes(time):
    hours = time // 100    # Extract hours
    minutes = time % 100   # Extract minutes
    return hours * 60 + minutes

# Apply the function to each time column
df['CRSDepTime'] = df['CRSDepTime'].apply(time_to_minutes)
df['CRSArrTime'] = df['CRSArrTime'].apply(time_to_minutes)

In [134]:
cut_off_date = pd.to_datetime('2024-08-25')

In [136]:
data = df[df['DateObjects'] >= cut_off_date]

In [138]:
data.columns

Index(['DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', 'DateObjects'],
      dtype='object')

In [140]:
data = data.drop(columns = ['OriginAirportID', 'OriginWac', 'DestAirportID', 'DestWac', 'DepTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay','tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', ], axis = 1)

In [142]:
data = data.drop(columns = ['CancellationCode_encoded'], axis = 1)

In [24]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

In [58]:
model = LpProblem("Minimize_Flight_Delays", LpMinimize)

# Create variables for departure delays and turnaround times
departure_delay = {i: LpVariable(f"departure_delay_{i}", lowBound=0) for i in data.index}
turnaround_time = {i: LpVariable(f"turnaround_time_{i}", lowBound=0) for i in data.index}

# Objective function: minimize the total departure delay
model += lpSum(departure_delay[i] for i in data.index), "Total_Delay"

# Constraints for sequential flights
for i in range(len(data) - 1):
    # Ensure flight i arrives before flight i+1 departs (including turnaround time)
    model += (data['CRSArrTime'][i] + turnaround_time[i] + departure_delay[i] 
              <= data['CRSArrTime'][i+1] + departure_delay[i+1]), f"Sequential_{i}"

# Minimum turnaround time constraint (e.g., 1 hour)
min_turnaround = 1  # Minimum 1 hour
for i in data.index:
    model += turnaround_time[i] >= min_turnaround, f"Turnaround_{i}"


In [38]:
model.solve()

# Output results
for i in data.index:
    print(f"Flight {data['Flight_Number_Reporting_Airline'][i]} delay: {departure_delay[i].value()} min")

Flight 5440 delay: 0.0 min
Flight 1272 delay: 0.0 min
Flight 2456 delay: 0.0 min
Flight 4684 delay: 705.0 min
Flight 2377 delay: 336.0 min
Flight 2354 delay: 0.0 min
Flight 1681 delay: 185.0 min
Flight 779 delay: 85.0 min
Flight 848 delay: 304.0 min
Flight 980 delay: 788.0 min
Flight 3121 delay: 975.0 min
Flight 3319 delay: 479.0 min
Flight 2900 delay: 76.0 min
Flight 4551 delay: 736.0 min
Flight 842 delay: 322.0 min
Flight 3319 delay: 1285.0 min
Flight 2077 delay: 1217.0 min
Flight 4079 delay: 1324.0 min
Flight 3695 delay: 0.0 min
Flight 2376 delay: 173.0 min
Flight 4933 delay: 1404.0 min
Flight 1142 delay: 591.0 min
Flight 1152 delay: 1429.0 min
Flight 465 delay: 3.0 min
Flight 763 delay: 136.0 min
Flight 102 delay: 23.0 min
Flight 4416 delay: 423.0 min
Flight 5490 delay: 1131.0 min
Flight 3406 delay: 1194.0 min
Flight 4338 delay: 1201.0 min
Flight 3019 delay: 1189.0 min
Flight 778 delay: 217.0 min
Flight 383 delay: 1340.0 min
Flight 246 delay: 1411.0 min
Flight 467 delay: 603.0 min


In [30]:
data.reset_index(drop=True, inplace=True)

In [46]:
from pulp import value

# Assuming 'model' is solved and 'data' contains original scheduled delays

# 1. Total Delay Reduction
initial_total_delay = data['DepDelayMinutes'].sum()  # Sum of original delays
optimized_total_delay = sum(value(departure_delay[i]) for i in data.index)
delay_reduction = initial_total_delay - optimized_total_delay

print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay: {optimized_total_delay} mins")
print(f"Total Delay Reduction: {delay_reduction} mins")

# 2. On-Time Performance (assuming 'on time' means no more than 15 minutes of delay)
on_time_threshold = 15  # 15 minutes in hours
on_time_flights = sum(1 for i in data.index if value(departure_delay[i]) <= on_time_threshold)
on_time_percentage = (on_time_flights / len(data)) * 100

print(f"On-Time Performance: {on_time_percentage:.2f}%")

Initial Total Delay: 62327.0 mins
Optimized Total Delay: 1006433.0 mins
Total Delay Reduction: -944106.0 mins
On-Time Performance: 0.99%


In [62]:
!pip install deap

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
      --------------------------------------- 0.0/1.1 MB 217.9 kB/s eta 0:00:05
      --------------------------------------- 0.0/1.1 MB 217.9 kB/s eta 0:00:05
     - -------------------------------------- 0.0/1.1 MB 217.9 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.1 MB 273.8 kB/s eta 0:00:04
     --- ------------------------------------ 0.1/1.1 MB 393.8 kB/s eta 0:00:03
     -------- ------------------------------- 0.2/1.1 MB 801.7 kB/s eta 0:00:02
     ------------------- -------------------- 0.5/1.1 MB 1.5 MB/s eta 0:00:01
     -------------------------------------- - 1.0/1.1 MB 2.7 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for deap: filename=deap-1.

In [120]:
from deap import base, creator, tools, algorithms
import random

# Set up the initial data and parameters
initial_delays = list(data['DepDelayMinutes'])  # Assume initial delays from data
num_flights = len(initial_delays)
delay_threshold = 30  # 30 minutes threshold for penalty

# Define fitness function to minimize total delay with penalty
def evaluate(individual):
    total_delay = 0
    for i in range(num_flights):
        delay = individual[i]
        # Add penalty for delays exceeding the threshold
        if delay > delay_threshold:
            delay += delay   # Example penalty factor for larger delays
        total_delay += delay
    return total_delay,

In [122]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize total delay
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_delay", random.uniform, 0, 2)  # Delay range from 0 to 2 hours
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_delay, n=num_flights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Crossover operation
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.1)  # Mutation operation
toolbox.register("select", tools.selTournament, tournsize=3)

# Initialize and evolve population
population = toolbox.population(n=50)
num_generations = 30

result = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, verbose=False)


C:\Users\kopal\anaconda\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\kopal\anaconda\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [124]:
best_individual = tools.selBest(population, k=1)[0]
optimized_delays = list(best_individual)

# Print results
print(f"Initial Total Delay: {sum(initial_delays)} min")
print(f"Optimized Total Delay (Genetic Algorithm): {evaluate(best_individual)[0]} min")

Initial Total Delay: 7609.0 min
Optimized Total Delay (Genetic Algorithm): -101.31673396893552 min


In [144]:
data['DepDelayMinutes'].sum()

58166.0

In [89]:
data['optimized_delay_ga'] = optimized_delays

# Calculate the rescheduled departure time based on optimized delays
data['rescheduled_departure_ga'] = data['CRSDepTime'] + data['optimized_delay_ga']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_ga', 'rescheduled_departure_ga']])

        DOT_ID_Reporting_Airline  Flight_Number_Reporting_Airline  CRSDepTime  \
359022                     19977                             2000        1086   
359671                     19805                             1927         588   
370315                     19805                             2361        1144   
360625                     19790                              405         430   
374388                     19393                              738        1280   
...                          ...                              ...         ...   
358053                     19805                              813         693   
360579                     19790                             2681         719   
372777                     19790                              820        1050   
367296                     19805                              615         600   
369902                     20363                             4956         840   

        optimized_delay_ga 

C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\1637929296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['optimized_delay_ga'] = optimized_delays
C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\1637929296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rescheduled_departure_ga'] = data['CRSDepTime'] + data['optimized_delay_ga']


In [97]:
!pip install simanneal

In [55]:
from simanneal import Annealer

In [91]:
class FlightDelayOptimizer(Annealer):
    def __init__(self, initial_state):
        super().__init__(initial_state)
    
    # Objective function: total delay with penalty for large delays
    def energy(self):
        total_delay = 0
        for delay in self.state:
            if delay > delay_threshold:
                delay += delay * 0.5  # Apply penalty
            total_delay += delay
        return total_delay

    # Define a move: slightly adjust delays
    def move(self):
        i = random.randint(0, len(self.state) - 1)
        self.state[i] += random.uniform(-0.1, 0.1)  # Adjust delay slightly
        self.state[i] = max(0, self.state[i]) 

In [93]:
initial_state = list(data['CRSDepTime'])

# Set up the annealer
optimizer = FlightDelayOptimizer(initial_state)
optimizer.steps = 10000  # Number of iterations
optimizer.Tmax = 2.0     # Starting temperature
optimizer.Tmin = 0.1     # Ending temperature

# Run simulated annealing
state, optimized_total_delay = optimizer.anneal()

# Print results
print(f"Initial Total Delay: {sum(initial_state)} mins")
print(f"Optimized Total Delay (Simulated Annealing): {optimized_total_delay} mins")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.10000    1153048.88    87.00%    55.00%     0:00:09     0:00:00

Initial Total Delay: 768765 mins
Optimized Total Delay (Simulated Annealing): 1153048.881597233 mins


In [94]:
data['optimized_delay_sa'] = state

# Calculate the rescheduled departure time based on optimized delays
data['rescheduled_departure_sa'] = data['CRSDepTime'] + data['optimized_delay_sa']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_sa', 'rescheduled_departure_sa']])

        DOT_ID_Reporting_Airline  Flight_Number_Reporting_Airline  CRSDepTime  \
359022                     19977                             2000        1086   
359671                     19805                             1927         588   
370315                     19805                             2361        1144   
360625                     19790                              405         430   
374388                     19393                              738        1280   
...                          ...                              ...         ...   
358053                     19805                              813         693   
360579                     19790                             2681         719   
372777                     19790                              820        1050   
367296                     19805                              615         600   
369902                     20363                             4956         840   

        optimized_delay_sa 

C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\1810452668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['optimized_delay_sa'] = state
C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\1810452668.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rescheduled_departure_sa'] = data['CRSDepTime'] + data['optimized_delay_sa']


In [126]:
import random

# Parameters
num_flights = len(data)
delay_threshold = 30  # 30 minutes in hours
penalty_factor = 1.5
turnaround_time = 15  # 15 minutes in hours
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
episodes = 1000

# Initialize Q-table with zeros
Q_table = np.zeros((num_flights, 10))

In [128]:
def calculate_reward(current_delay, next_delay):
    # Penalize larger delays
    reward = -next_delay
    if next_delay > delay_threshold:
        reward -= penalty_factor * (next_delay - delay_threshold)
    return reward

# Initialize a delay schedule for flights
initial_delays = list(data['DepDelayMinutes'])
state = 0  # Starting at the first flight

# Q-learning algorithm
for episode in range(episodes):
    state = 0  # Reset to the first flight for each episode
    total_delay = 0
    
    for i in range(num_flights - 1):
        # Choose an action (explore or exploit)
        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, 9)  # Randomly choose a delay adjustment
        else:
            action = np.argmax(Q_table[state])  # Choose the best action from the Q-table
        
        # Apply action (adjust delay)
        delay_adjustment = action * 5  # Example delay adjustment increment (0 to 0.9 hours)
        next_delay = initial_delays[state] + delay_adjustment
        
        # Calculate reward
        reward = calculate_reward(initial_delays[state], next_delay)
        
        # Move to the next state
        next_state = state + 1
        total_delay += next_delay
        
        # Update Q-value
        Q_table[state, action] = Q_table[state, action] + alpha * (
            reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action]
        )
        
        # Update state
        state = next_state

In [129]:
optimized_delays_rl = [initial_delays[i] + np.argmax(Q_table[i]) * 0.1 for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_rl'] = optimized_delays_rl
data['rescheduled_departure_rl'] = data['CRSDepTime'] + data['optimized_delay_rl']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_rl', 'rescheduled_departure_rl']])

        DOT_ID_Reporting_Airline  Flight_Number_Reporting_Airline  CRSDepTime  \
359264                     19805                             1671        1165   
365795                     20397                             5256         679   
357612                     19805                             2631         840   
370974                     19790                             1324        1327   
375500                     19805                              842        1022   
...                          ...                              ...         ...   
358053                     19805                              813         693   
360579                     19790                             2681         719   
372777                     19790                              820        1050   
367296                     19805                              615         600   
369902                     20363                             4956         840   

        optimized_delay_rl 

C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\3940145359.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['optimized_delay_rl'] = optimized_delays_rl
C:\Users\kopal\AppData\Local\Temp\ipykernel_6440\3940145359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rescheduled_departure_rl'] = data['CRSDepTime'] + data['optimized_delay_rl']


In [130]:
initial_total_delay = sum(initial_delays)
optimized_total_delay_rl = sum(optimized_delays_rl)

# Print comparison
print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay (Reinforcement Learning): {optimized_total_delay_rl} mins")

# Calculate percentage of on-time flights
on_time_rl = len([d for d in optimized_delays_rl if d <= delay_threshold]) / num_flights * 100
print(f"On-Time Performance (RL): {on_time_rl}%")

Initial Total Delay: 7609.0 mins
Optimized Total Delay (Reinforcement Learning): 7631.100000000002 mins
On-Time Performance (RL): 33.33333333333333%


In [147]:
cut_off_date = pd.to_datetime('2024-01-01')

In [148]:
data = df[df['DateObjects'] >= cut_off_date]

In [149]:
num_flights = len(data)
delay_threshold = 0.5  # 30 minutes in hours
penalty_factor = 1.5
turnaround_time = 0.25  # 15 minutes in hours
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
episodes = 1000

# Initialize Q-table with zeros
Q_table = np.zeros((num_flights, 10))

In [150]:
def calculate_reward(current_delay, next_delay):
    # Penalize larger delays
    reward = -next_delay
    if next_delay > delay_threshold:
        reward -= penalty_factor * (next_delay - delay_threshold)
    return reward

# Initialize a delay schedule for flights
initial_delays = list(data['DepDelayMinutes'])
state = 0  # Starting at the first flight

# Q-learning algorithm
for episode in range(episodes):
    state = 0  # Reset to the first flight for each episode
    total_delay = 0
    
    for i in range(num_flights - 1):
        # Choose an action (explore or exploit)
        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, 9)  # Randomly choose a delay adjustment
        else:
            action = np.argmax(Q_table[state])  # Choose the best action from the Q-table
        
        # Apply action (adjust delay)
        delay_adjustment = action * 0.1  # Example delay adjustment increment (0 to 0.9 hours)
        next_delay = initial_delays[state] + delay_adjustment
        
        # Calculate reward
        reward = calculate_reward(initial_delays[state], next_delay)
        
        # Move to the next state
        next_state = state + 1
        total_delay += next_delay
        
        # Update Q-value
        Q_table[state, action] = Q_table[state, action] + alpha * (
            reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action]
        )
        
        # Update state
        state = next_state

In [151]:
optimized_delays_rl = [initial_delays[i] + np.argmax(Q_table[i]) * 0.1 for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_rl'] = optimized_delays_rl
data['rescheduled_departure_rl'] = data['CRSDepTime'] + data['optimized_delay_rl']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_rl', 'rescheduled_departure_rl']])

        DOT_ID_Reporting_Airline  Flight_Number_Reporting_Airline  CRSDepTime  \
12670                      19393                             2381        1940   
1039                       19393                             1906        2045   
5789                       19690                               25         745   
9521                       19930                              622        1859   
16416                      19977                             1102        1000   
...                          ...                              ...         ...   
358053                     19805                              813        1133   
360579                     19790                             2681        1159   
372777                     19790                              820        1730   
367296                     19805                              615        1000   
369902                     20363                             4956        1400   

        optimized_delay_rl 

C:\Users\kopal\AppData\Local\Temp\ipykernel_27448\3940145359.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['optimized_delay_rl'] = optimized_delays_rl
C:\Users\kopal\AppData\Local\Temp\ipykernel_27448\3940145359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rescheduled_departure_rl'] = data['CRSDepTime'] + data['optimized_delay_rl']


In [152]:
initial_total_delay = sum(initial_delays)
optimized_total_delay_rl = sum(optimized_delays_rl)

# Print comparison
print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay (Reinforcement Learning): {optimized_total_delay_rl} mins")

# Calculate percentage of on-time flights
on_time_rl = len([d for d in optimized_delays_rl if d <= delay_threshold]) / num_flights * 100
print(f"On-Time Performance (RL): {on_time_rl}%")

Initial Total Delay: 2278199.0 mins
Optimized Total Delay (Reinforcement Learning): 2292563.0000000726 mins
On-Time Performance (RL): 5.7720953326713005%


In [178]:
cut_off_date = pd.to_datetime('2024-08-25')

In [180]:
data = df[df['DateObjects'] >= cut_off_date]

In [182]:
import tensorflow as tf
from collections import deque

# Define parameters
num_flights = len(data)
delay_threshold = 0.5  # 30 minutes in hours
episodes = 1000
gamma = 0.99  # Discount factor
epsilon_start = 1.0  # Start of exploration
epsilon_end = 0.1    # End of exploration
epsilon_decay = 0.995  # Decay rate of exploration
learning_rate = 0.001
memory_size = 2000

In [184]:
def build_model(input_shape, action_space):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_shape=input_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=memory_size)
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.model = build_model((state_size,), action_size)
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)  # Explore
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # Exploit

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize DQN agent
agent = DQNAgent(state_size=num_flights, action_size=10)  # Assuming 10 possible delay actions

# Training the DQN agent
batch_size = 32
for episode in range(episodes):
    state = np.array(initial_delays)  # Current state (delays)
    state = np.reshape(state, [1, num_flights])  # Reshape for model input
    total_delay = 0

    for time in range(num_flights - 1):
        action = agent.act(state)  # Select action
        delay_adjustment = action * 0.1  # Example delay adjustment
        next_delay = initial_delays[time] + delay_adjustment
        reward = calculate_reward(initial_delays[time], next_delay)  # Reward calculation
        total_delay += next_delay
        
        # Define next state
        next_state = np.array(initial_delays) 
        next_state = np.reshape(next_state, [1, num_flights])
        
        done = (time == num_flights - 2)  # End of episode
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state  # Move to next state

    agent.replay(batch_size)  # Replay experience
    print(f"Episode: {episode + 1}/{episodes}, Total Delay: {total_delay:.2f}")

# Extract optimized delays
optimized_delays_dqn = [[initial_delays[i] + np.argmax(agent.model.predict(np.array(initial_delays).reshape(1, num_flights)))] for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_dqn'] = optimized_delays_dqn
data['rescheduled_departure_dqn'] = data['CRSDepTime'] + data['optimized_delay_dqn']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_dqn', 'rescheduled_departure_dqn']])

C:\Users\kopal\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: cannot reshape array of size 32224 into shape (1,838)

In [ ]:
initial_total_delay = sum(initial_delays)
optimized_total_delay_dqn = sum(optimized_delays_dqn)

# Print comparison
print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay (DQN): {optimized_total_delay_dqn} mins")

# Calculate percentage of on-time flights
on_time_dqn = len([d for d in optimized_delays_dqn if d <= delay_threshold]) / num_flights * 100
print(f"On-Time Performance (DQN): {on_time_dqn}%")

In [ ]:
cut_off_date = pd.to_datetime('2024-08-01')

In [ ]:
data = df[df['DateObjects'] >= cut_off_date]

In [ ]:
import tensorflow as tf
from collections import deque

# Define parameters
num_flights = len(data)
delay_threshold = 0.5  # 30 minutes in hours
episodes = 1000
gamma = 0.99  # Discount factor
epsilon_start = 1.0  # Start of exploration
epsilon_end = 0.1    # End of exploration
epsilon_decay = 0.995  # Decay rate of exploration
learning_rate = 0.001
memory_size = 2000

In [ ]:
def build_model(input_shape, action_space):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_shape=input_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=memory_size)
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.model = build_model((state_size,), action_size)
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)  # Explore
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # Exploit

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize DQN agent
agent = DQNAgent(state_size=num_flights, action_size=10)  # Assuming 10 possible delay actions

# Training the DQN agent
batch_size = 32
for episode in range(episodes):
    state = np.array(initial_delays)  # Current state (delays)
    state = np.reshape(state, [1, num_flights])  # Reshape for model input
    total_delay = 0

    for time in range(num_flights - 1):
        action = agent.act(state)  # Select action
        delay_adjustment = action * 0.1  # Example delay adjustment
        next_delay = initial_delays[time] + delay_adjustment
        reward = calculate_reward(initial_delays[time], next_delay)  # Reward calculation
        total_delay += next_delay
        
        # Define next state
        next_state = np.array(initial_delays) 
        next_state = np.reshape(next_state, [1, num_flights])
        
        done = (time == num_flights - 2)  # End of episode
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state  # Move to next state

    agent.replay(batch_size)  # Replay experience
    print(f"Episode: {episode + 1}/{episodes}, Total Delay: {total_delay:.2f}")

# Extract optimized delays
optimized_delays_dqn = [initial_delays[i] + np.argmax(agent.model.predict(np.array(initial_delays).reshape(1, num_flights)))] for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_dqn'] = optimized_delays_dqn
data['rescheduled_departure_dqn'] = data['CRSDepTime'] + data['optimized_delay_dqn']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_dqn', 'rescheduled_departure_dqn']])

In [ ]:
initial_total_delay = sum(initial_delays)
optimized_total_delay_dqn = sum(optimized_delays_dqn)

# Print comparison
print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay (DQN): {optimized_total_delay_dqn} mins")

# Calculate percentage of on-time flights
on_time_dqn = len([d for d in optimized_delays_dqn if d <= delay_threshold]) / num_flights * 100
print(f"On-Time Performance (DQN): {on_time_dqn}%")

In [ ]:
cut_off_date = pd.to_datetime('2024-01-01')

In [ ]:
data = df[df['DateObjects'] >= cut_off_date]

In [ ]:
num_flights = len(data)
delay_threshold = 0.5  # 30 minutes in hours
episodes = 1000
gamma = 0.99  # Discount factor
epsilon_start = 1.0  # Start of exploration
epsilon_end = 0.1    # End of exploration
epsilon_decay = 0.995  # Decay rate of exploration
learning_rate = 0.001
memory_size = 2000

In [ ]:
def build_model(input_shape, action_space):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_shape=input_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=memory_size)
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.model = build_model((state_size,), action_size)
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)  # Explore
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # Exploit

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize DQN agent
agent = DQNAgent(state_size=num_flights, action_size=10)  # Assuming 10 possible delay actions

# Training the DQN agent
batch_size = 32
for episode in range(episodes):
    state = np.array(initial_delays)  # Current state (delays)
    state = np.reshape(state, [1, num_flights])  # Reshape for model input
    total_delay = 0

    for time in range(num_flights - 1):
        action = agent.act(state)  # Select action
        delay_adjustment = action * 0.1  # Example delay adjustment
        next_delay = initial_delays[time] + delay_adjustment
        reward = calculate_reward(initial_delays[time], next_delay)  # Reward calculation
        total_delay += next_delay
        
        # Define next state
        next_state = np.array(initial_delays) 
        next_state = np.reshape(next_state, [1, num_flights])
        
        done = (time == num_flights - 2)  # End of episode
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state  # Move to next state

    agent.replay(batch_size)  # Replay experience
    print(f"Episode: {episode + 1}/{episodes}, Total Delay: {total_delay:.2f}")

# Extract optimized delays
optimized_delays_dqn = [initial_delays[i] + np.argmax(agent.model.predict(np.array(initial_delays).reshape(1, num_flights)))] for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_dqn'] = optimized_delays_dqn
data['rescheduled_departure_dqn'] = data['CRSDepTime'] + data['optimized_delay_dqn']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_dqn', 'rescheduled_departure_dqn']])

In [ ]:
initial_total_delay = sum(initial_delays)
optimized_total_delay_dqn = sum(optimized_delays_dqn)

# Print comparison
print(f"Initial Total Delay: {initial_total_delay} mins")
print(f"Optimized Total Delay (DQN): {optimized_total_delay_dqn} mins")

# Calculate percentage of on-time flights
on_time_dqn = len([d for d in optimized_delays_dqn if d <= delay_threshold]) / num_flights * 100
print(f"On-Time Performance (DQN): {on_time_dqn}%")

In [233]:
cut_off_date = pd.to_datetime('2024-01-01')

In [234]:
data = df[df['DateObjects'] >= cut_off_date]

In [218]:
from tensorflow.keras import layers

# Define parameters
num_flights = len(data)
delay_threshold = 0.5  # 30 minutes in hours
epochs = 10
steps_per_epoch = 1000
gamma = 0.99  # Discount factor
clip_ratio = 0.2  # PPO clip ratio
learning_rate = 0.001
batch_size = 64

In [220]:
def build_model(input_shape, action_space):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(action_space, activation='softmax')(x)
    value_output = layers.Dense(1)(x)
    return tf.keras.Model(inputs, [outputs, value_output])

# PPO Agent
class PPOAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.model = build_model((state_size,), action_size)
        self.old_model = build_model((state_size,), action_size)  # For clipping
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    def get_action(self, state):
        state = state.reshape(1, self.state_size)
        action_probs, _ = self.model(state)
        action = np.random.choice(self.action_size, p=action_probs.numpy()[0])
        return action

    def train(self, states, actions, rewards, next_states, dones):
        # Compute advantages and targets
        _, value_targets = self.model.predict(states)
        _, next_value_targets = self.model.predict(next_states)

        targets = rewards + (1 - dones) * gamma * next_value_targets.flatten()
        advantages = targets - value_targets.flatten()

        # Perform PPO update
        with tf.GradientTape() as tape:
            action_probs, value_preds = self.model(states)
            action_probs_old, _ = self.old_model(states)
            
            # Calculate the policy loss
            prob_ratio = tf.reduce_sum(action_probs * tf.one_hot(actions, self.action_size), axis=1) / \
                         tf.reduce_sum(action_probs_old * tf.one_hot(actions, self.action_size), axis=1)
            p1 = prob_ratio * advantages
            p2 = tf.clip_by_value(prob_ratio, 1 - clip_ratio, 1 + clip_ratio) * advantages
            policy_loss = -tf.reduce_mean(tf.minimum(p1, p2))

            # Calculate the value loss
            value_loss = tf.reduce_mean(tf.square(targets - value_preds))

            # Total loss
            total_loss = policy_loss + value_loss

        # Apply gradients
        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # Update old model
        self.old_model.set_weights(self.model.get_weights())

# Initialize PPO agent
agent = PPOAgent(state_size=num_flights, action_size=10)  # Assuming 10 possible delay actions

In [222]:
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        state = np.array(initial_delays)
        done = False
        total_reward = 0
        states, actions, rewards, next_states, dones = [], [], [], [], []

        while not done:
            action = agent.get_action(state)  # Select action
            delay_adjustment = action * 0.1  # Example delay adjustment
            next_delay = state[step % num_flights] + delay_adjustment
            
            reward = calculate_reward(state[step % num_flights], next_delay)  # Reward calculation
            total_reward += reward
            
            next_state = np.array(initial_delays)  # Reshape for the next flight
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(1 if step == num_flights - 1 else 0)  # End of episode if last flight
            
            state = next_state  # Move to next state

        # Train agent with collected data
        agent.train(np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones))

    print(f"Epoch: {epoch + 1}/{epochs}, Total Reward: {total_reward:.2f}")

# Extract optimized delays
optimized_delays_ppo = [[initial_delays[i] + np.argmax(agent.model.predict(np.array(initial_delays).reshape(1, num_flights)))] for i in range(num_flights)]

# Update DataFrame with the optimized delays
data['optimized_delay_ppo'] = optimized_delays_ppo
data['rescheduled_departure_ppo'] = data['CRSDepTime'] + data['optimized_delay_ppo']

# Print the updated DataFrame
print(data[['DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'CRSDepTime', 'optimized_delay_ppo', 'rescheduled_departure_ppo']])

ValueError: cannot reshape array of size 32224 into shape (1,4322)

In [224]:
NUM_FLIGHTS = 4322  # Number of flights to schedule
MAX_GENERATIONS = 100  # Maximum number of generations
POPULATION_SIZE = 20  # Size of the population
MUTATION_RATE = 0.1  # Probability of mutation
TOURNAMENT_SIZE = 3  # Size of the tournament for selection

# Example flight data: [flight_number, scheduled_time, delay]
flights = [[i, random.randint(0, 100), random.randint(0, 30)] for i in range(NUM_FLIGHTS)]


In [226]:
def calculate_fitness(schedule):
    total_delay = 0
    current_time = 0
    
    for flight in schedule:
        flight_info = flights[flight]
        scheduled_time = flight_info[1]
        delay = flight_info[2]
        
        # Update current time considering previous delays
        current_time += delay
        total_delay += current_time - scheduled_time  # Penalty for being late

    return total_delay

# Generate initial population
def generate_population(size):
    return [random.sample(range(NUM_FLIGHTS), NUM_FLIGHTS) for _ in range(size)]


In [228]:
def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    tournament.sort(key=lambda x: calculate_fitness(x))
    return tournament[0]

# Crossover (Single Point Crossover)
def crossover(parent1, parent2):
    point = random.randint(1, NUM_FLIGHTS - 1)
    child = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    return child

# Mutation (Swap Mutation)
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(NUM_FLIGHTS), 2)
        schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]


In [230]:
def genetic_algorithm():
    population = generate_population(POPULATION_SIZE)

    for generation in range(MAX_GENERATIONS):
        new_population = []

        for _ in range(POPULATION_SIZE):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population

    # Evaluate final population
    fitness_scores = [calculate_fitness(schedule) for schedule in population]
    best_schedule = population[np.argmin(fitness_scores)]
    best_fitness = min(fitness_scores)

    return best_schedule, best_fitness

In [232]:
best_schedule, best_fitness = genetic_algorithm()

print("Best Schedule:", best_schedule)
print("Total Delay (Fitness):", best_fitness)

Best Schedule: [2290, 1618, 4215, 203, 1495, 3517, 446, 1583, 966, 4132, 2470, 2315, 2121, 2294, 2502, 2733, 4043, 1072, 2288, 2944, 3793, 973, 1774, 3961, 1121, 3436, 3630, 403, 2001, 507, 2232, 218, 290, 195, 3025, 893, 1490, 731, 1787, 1789, 1630, 3352, 3071, 3642, 1662, 1574, 3684, 858, 2440, 3064, 615, 2997, 1699, 735, 951, 2539, 1040, 2712, 4058, 2044, 2857, 1431, 2620, 1227, 4086, 129, 4240, 1648, 472, 1432, 2247, 2177, 3267, 986, 418, 393, 2263, 2408, 642, 896, 1029, 957, 388, 2425, 3568, 2441, 2720, 2101, 2105, 2100, 3408, 1238, 1694, 2427, 1045, 1380, 2693, 1912, 317, 1175, 639, 2651, 1858, 3122, 2158, 1153, 3268, 911, 399, 3145, 3378, 3928, 108, 1632, 3240, 361, 3050, 1738, 168, 2363, 3819, 4183, 2090, 2053, 3985, 1409, 2102, 1472, 1890, 3691, 2606, 3353, 1480, 400, 366, 2972, 584, 1693, 1123, 2234, 2934, 1507, 2354, 4307, 3976, 776, 1506, 2082, 2959, 3396, 2241, 1314, 1698, 1819, 665, 1788, 4269, 2561, 1741, 1010, 3405, 3682, 4100, 3502, 16, 481, 3636, 4315, 2770, 4255, 415

In [235]:
best_schedule, best_fitness = genetic_algorithm()

print("Best Schedule:", best_schedule)
print("Total Delay (Fitness):", best_fitness)

KeyboardInterrupt: 

In [246]:
cut_off_date = pd.to_datetime('2024-08-25')
data = df[df['DateObjects'] >= cut_off_date]

In [248]:
NUM_FLIGHTS = len(data)
MAX_GENERATIONS = 100  # Maximum number of generations
POPULATION_SIZE = 20  # Size of the population
MUTATION_RATE = 0.1  # Probability of mutation
TOURNAMENT_SIZE = 3  # Size of the tournament for selection

# Fitness function
def calculate_fitness(schedule):
    total_delay = 0
    current_time = 0
    
    for flight_idx in schedule:
        flight_info = data.iloc[flight_idx]
        scheduled_time = flight_info['CRSDepTime']
        delay = flight_info['DepDelayMinutes']
        
        # Update current time considering previous delays
        current_time += delay
        total_delay += max(0, current_time - scheduled_time)  # Penalty for being late

    return total_delay

In [250]:
def generate_population(size):
    return [random.sample(range(NUM_FLIGHTS), NUM_FLIGHTS) for _ in range(size)]

# Selection (Tournament Selection)
def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    tournament.sort(key=lambda x: calculate_fitness(x))
    return tournament[0]

# Crossover (Single Point Crossover)
def crossover(parent1, parent2):
    point = random.randint(1, NUM_FLIGHTS - 1)
    child = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    return child

# Mutation (Swap Mutation)
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(NUM_FLIGHTS), 2)
        schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]

In [252]:
def genetic_algorithm():
    population = generate_population(POPULATION_SIZE)

    for generation in range(MAX_GENERATIONS):
        new_population = []

        for _ in range(POPULATION_SIZE):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population

    # Evaluate final population
    fitness_scores = [calculate_fitness(schedule) for schedule in population]
    best_schedule = population[np.argmin(fitness_scores)]
    best_fitness = min(fitness_scores)

    return best_schedule, best_fitness


In [254]:
best_schedule, best_fitness = genetic_algorithm()

# Show the results
print("Best Schedule (Flight Indices):", best_schedule)
print("Total Delay (Fitness):", best_fitness)

# Optional: Show the best schedule with corresponding flight details
best_flights = data.iloc[best_schedule]
print("Best Flight Schedule:\n", best_flights)

Best Schedule (Flight Indices): [604, 75, 459, 738, 40, 181, 228, 76, 7, 629, 280, 227, 213, 461, 382, 703, 560, 505, 405, 159, 352, 543, 644, 681, 390, 178, 671, 161, 128, 335, 198, 283, 680, 381, 80, 37, 62, 480, 361, 565, 440, 96, 394, 344, 430, 153, 700, 9, 33, 310, 530, 174, 489, 156, 498, 169, 805, 529, 140, 101, 729, 319, 0, 518, 85, 102, 48, 16, 49, 13, 327, 603, 126, 350, 264, 336, 94, 114, 8, 172, 349, 223, 621, 139, 748, 87, 57, 478, 770, 630, 17, 53, 194, 578, 582, 120, 343, 123, 241, 734, 684, 401, 720, 34, 362, 747, 528, 726, 538, 221, 807, 553, 701, 639, 548, 190, 688, 347, 789, 661, 417, 182, 122, 576, 779, 756, 722, 767, 766, 570, 201, 820, 332, 103, 708, 502, 176, 482, 653, 312, 32, 763, 804, 129, 107, 510, 451, 593, 473, 495, 154, 647, 652, 679, 402, 428, 371, 341, 424, 366, 497, 508, 289, 777, 271, 323, 150, 36, 252, 801, 299, 238, 623, 98, 81, 71, 737, 282, 63, 148, 340, 414, 507, 284, 259, 667, 322, 615, 568, 833, 135, 687, 444, 4, 23, 51, 189, 218, 601, 408, 253,

In [255]:
cut_off_date = pd.to_datetime('2024-08-01')
data = df[df['DateObjects'] >= cut_off_date]

In [256]:
NUM_FLIGHTS = len(data)
MAX_GENERATIONS = 100  # Maximum number of generations
POPULATION_SIZE = 20  # Size of the population
MUTATION_RATE = 0.1  # Probability of mutation
TOURNAMENT_SIZE = 3  # Size of the tournament for selection

# Fitness function
def calculate_fitness(schedule):
    total_delay = 0
    current_time = 0
    
    for flight_idx in schedule:
        flight_info = data.iloc[flight_idx]
        scheduled_time = flight_info['CRSDepTime']
        delay = flight_info['DepDelayMinutes']
        
        # Update current time considering previous delays
        current_time += delay
        total_delay += max(0, current_time - scheduled_time)  # Penalty for being late

    return total_delay

def generate_population(size):
    return [random.sample(range(NUM_FLIGHTS), NUM_FLIGHTS) for _ in range(size)]

# Selection (Tournament Selection)
def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    tournament.sort(key=lambda x: calculate_fitness(x))
    return tournament[0]

# Crossover (Single Point Crossover)
def crossover(parent1, parent2):
    point = random.randint(1, NUM_FLIGHTS - 1)
    child = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    return child

# Mutation (Swap Mutation)
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(NUM_FLIGHTS), 2)
        schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]

In [257]:
def genetic_algorithm():
    population = generate_population(POPULATION_SIZE)

    for generation in range(MAX_GENERATIONS):
        new_population = []

        for _ in range(POPULATION_SIZE):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population

    # Evaluate final population
    fitness_scores = [calculate_fitness(schedule) for schedule in population]
    best_schedule = population[np.argmin(fitness_scores)]
    best_fitness = min(fitness_scores)

    return best_schedule, best_fitness

best_schedule, best_fitness = genetic_algorithm()

# Show the results
print("Best Schedule (Flight Indices):", best_schedule)
print("Total Delay (Fitness):", best_fitness)

# Optional: Show the best schedule with corresponding flight details
best_flights = data.iloc[best_schedule]
print("Best Flight Schedule:\n", best_flights)

Best Schedule (Flight Indices): [3182, 1563, 3500, 3424, 571, 200, 1540, 3743, 1844, 3992, 1548, 950, 3472, 3955, 1047, 2408, 130, 234, 651, 1206, 555, 2975, 4156, 3125, 465, 1326, 767, 1638, 2938, 3341, 2396, 1297, 3228, 3320, 2065, 3388, 454, 1370, 4064, 2726, 2476, 738, 2233, 3401, 194, 771, 2027, 788, 75, 1343, 1095, 2604, 2461, 1002, 2251, 275, 1413, 931, 3769, 29, 1007, 1186, 4023, 1427, 1278, 2459, 3812, 1622, 155, 829, 1333, 310, 2132, 2824, 2374, 1316, 441, 1134, 1404, 1490, 229, 1663, 3826, 3580, 2679, 3930, 2700, 3563, 2142, 57, 3308, 1865, 4174, 1357, 2923, 885, 53, 1513, 4183, 3508, 99, 2502, 2543, 1392, 2302, 3409, 1443, 554, 4278, 810, 365, 2487, 1501, 2951, 3271, 1207, 3041, 793, 1578, 1777, 2427, 3707, 48, 3342, 2298, 338, 2968, 110, 868, 4061, 97, 3049, 2359, 2146, 2699, 3491, 4132, 205, 1510, 1525, 2464, 3056, 703, 3394, 3014, 3493, 1053, 2025, 2215, 2371, 1802, 3883, 612, 3488, 1041, 185, 3866, 1763, 1733, 3840, 2279, 1933, 3192, 31, 85, 3087, 3561, 3175, 2529, 592,

In [258]:
cut_off_date = pd.to_datetime('2024-01-01')
data = df[df['DateObjects'] >= cut_off_date]

In [ ]:
NUM_FLIGHTS = len(data)
MAX_GENERATIONS = 100  # Maximum number of generations
POPULATION_SIZE = 20  # Size of the population
MUTATION_RATE = 0.1  # Probability of mutation
TOURNAMENT_SIZE = 3  # Size of the tournament for selection

# Fitness function
def calculate_fitness(schedule):
    total_delay = 0
    current_time = 0
    
    for flight_idx in schedule:
        flight_info = data.iloc[flight_idx]
        scheduled_time = flight_info['CRSDepTime']
        delay = flight_info['DepDelayMinutes']
        
        # Update current time considering previous delays
        current_time += delay
        total_delay += max(0, current_time - scheduled_time)  # Penalty for being late

    return total_delay

def generate_population(size):
    return [random.sample(range(NUM_FLIGHTS), NUM_FLIGHTS) for _ in range(size)]

# Selection (Tournament Selection)
def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    tournament.sort(key=lambda x: calculate_fitness(x))
    return tournament[0]

# Crossover (Single Point Crossover)
def crossover(parent1, parent2):
    point = random.randint(1, NUM_FLIGHTS - 1)
    child = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    return child

# Mutation (Swap Mutation)
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(NUM_FLIGHTS), 2)
        schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]

def genetic_algorithm():
    population = generate_population(POPULATION_SIZE)

    for generation in range(MAX_GENERATIONS):
        new_population = []

        for _ in range(POPULATION_SIZE):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population

    # Evaluate final population
    fitness_scores = [calculate_fitness(schedule) for schedule in population]
    best_schedule = population[np.argmin(fitness_scores)]
    best_fitness = min(fitness_scores)

    return best_schedule, best_fitness

best_schedule, best_fitness = genetic_algorithm()

# Show the results
print("Best Schedule (Flight Indices):", best_schedule)
print("Total Delay (Fitness):", best_fitness)

# Optional: Show the best schedule with corresponding flight details
best_flights = data.iloc[best_schedule]
print("Best Flight Schedule:\n", best_flights)